# CLOUD PROBABILITY PRODUCT

In this notebook we explore how to create an annual cloud probability product using Sentinel-2 satellite imagery data. We describe how to use various parameters and configurations to obtain raw band data and product.

To calculate the sun exposure/sunniness/daylight (?) index, we would have to use many spatio-temporal variables, including the area cloudiness, or probability of retrieveing clouds in the particular point in space and time.

Py-STAC solution is based on the [carpentries guide](https://carpentries-incubator.github.io/geospatial-python/05-access-data.html).
This Notebook has been partially based on the [Sentinel Hub API example](https://sentinelhub-py.readthedocs.io/en/latest/examples/process_request.html).

### Dependencies
We use the base [Docker image](https://hub.docker.com/r/behzad89/geo-miniconda3) to run the Notebook and [helper functions](raster_utils.py) and also install [`pystac-client`](https://pystac-client.readthedocs.io/en/stable/tutorials/authentication.html) to access data.

### Auxiliary data
To assist in the computation, we'll use two small input datasets, [located here](data/):
- boundary of the Tyne and Wear area (Newcastle agglomeration) from [here](link?)
- British National Grid, which covers the whole UK from [here](https://github.com/OrdnanceSurvey/OS-British-National-Grids?tab=readme-ov-file)

~~The access token is obtained through Copernicus Data Ecosystem: https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html#:~:text=Client%20in%20your-,account%20settings,-.%20This%20is%20so~~

~~Here is the instruction to register a token: https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html~~

#### Py-stac solution

Let's configure our pystac client first, find out all available collections and stick to one of them. It has been found out that cloud probabilities are described by the Sentinel-2 Collection 1 L2A product.

Usually, the cloud probability products are available through Sentinel Hub API which is subject to significant access restrictions. Although many hubs provide free access to Sentinel-2 products, they do not include additional bands, such as a cloud probability (Copernicus Data Space Ecosystem, Microsoft Planetary). However, Element84 (Earth Search) provides access to cloud probability products to kept on AWS storage. For details, see the registry key ([here](https://registry.opendata.aws/sentinel-2-l2a-cogs/)) and STAC catalogue of the relevant Sentinel-2 collection([here](https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/collections/sentinel-2-c1-l2a)).

**TODO:** back-up plan if Element84 terminate access. Any limit rate for earth84?

In [5]:
# pip install pystac-client if not installed
from pystac_client import Client

api_url = "https://earth-search.aws.element84.com/v1"
# OTHER ENDPOINTS TO GET DATA
#api_url = "https://stac.dataspace.copernicus.eu/v1/" # NOTE: this doesn't contain cloud bands in the assets
# https://hub.openeo.org/ (not production ready)
client = Client.open(api_url)

collections = client.get_collections()
for collection in collections:
    print(collection)
collection = "sentinel-2-c1-l2a" # NOTE: do not use "sentinel-2-l2a" as it doesn't contain cloud probabilities

datetime = '2024-01-01/2024-01-31'

# NOTE: Connecting to client might freeze sometimes, kernel restart would help

<CollectionClient id=sentinel-2-pre-c1-l2a>
<CollectionClient id=cop-dem-glo-30>
<CollectionClient id=naip>
<CollectionClient id=cop-dem-glo-90>
<CollectionClient id=landsat-c2-l2>
<CollectionClient id=sentinel-2-l2a>
<CollectionClient id=sentinel-2-l1c>
<CollectionClient id=sentinel-2-c1-l2a>
<CollectionClient id=sentinel-1-grd>


### 1. CLOUD PROBABILITY ACCESS


We will download Sentinel-2 imagery of Tyne and Wear Area. Let's try with a just one 20-km tile of the area of interest.
These tiles have already been prepared.

The bounding box of this tile in `WGS84` coordinate system is `[54.933089, -1.689407, 55.114004, -1.374500]` (longitude and latitude coordinates of lower left and upper right corners).

![area_of_interest_tile](illustrations/area_of_interest_tile.png)

In [6]:
# PNG visualisation - another way
"""
from IPython.display import Image, display

display(
    Image(filename="illustrations/area_of_interest_tile.png, width=400),
    Image(filename="illustrations/nodata_issue_snow_legend.png", width=250)
)
"""

'\nfrom IPython.display import Image, display\n\ndisplay(\n    Image(filename="illustrations/area_of_interest_tile.png, width=400),\n    Image(filename="illustrations/nodata_issue_snow_legend.png", width=250)\n)\n'

In [7]:
from shapely.geometry import Polygon

# define polygon vertices as (lat, lon) tuples - correct for Shapely
tile = Polygon([
    (-1.689407, 54.933089),  # lat, lon swapped
    (-1.374500, 54.933089),
    (-1.374500, 55.114004),
    (-1.689407, 55.114004),
    (-1.689407, 54.933089)
])

search = client.search(
    collections=[collection],
    intersects=tile,
    datetime=datetime
)

print(f"Number of matched scenes: {search.matched()}")

# TODO - to write a function to transform the tile extent into Shapely polygon


Number of matched scenes: 64


For a tile 20x20 km in the UK we will usually have hundreds of acquisitions per year. For example, for the sample tile we got 713 items.

In [8]:
items = search.item_collection()
print(f"Number of items: {len(items)}")
print("-" * 20)

for item in items[:5]:
    print(item)
print("...")

Number of items: 64
--------------------
<Item id=S2B_T30UWF_20240131T111211_L2A>
<Item id=S2B_T30UXF_20240131T111211_L2A>
<Item id=S2B_T30UWG_20240131T111211_L2A>
<Item id=S2B_T30UXG_20240131T111211_L2A>
<Item id=S2A_T30UWF_20240129T112442_L2A>
...


For inspection, let's check out one of the scenes:

In [9]:
index = 20
if index > len(items):
    raise IndexError(f"You've chosen the item number which exceeds the total number of items {index}>{len(items)}")

item = items[index]
try:
    print(item.datetime)
    print(item.geometry)
    print(item.properties)
    print(item.properties.get('proj:code') or item.properties.get('proj:epsg'))
except Exception as e:
    print(f"Error checking item[{index}]: {e}")

2024-01-21 11:16:08.468000+00:00
{'type': 'Polygon', 'coordinates': [[[-1.4748970523669551, 55.037423174529614], [-1.9322767235365357, 54.055473781928036], [-1.3232267769574286, 54.04852390275896], [-1.2822817967468176, 55.034856954918745], [-1.4748970523669551, 55.037423174529614]]]}
{'created': '2024-01-22T00:52:27.333Z', 'platform': 'sentinel-2b', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 89.208281, 'proj:centroid': {'lat': 54.45889, 'lon': -1.52475}, 'mgrs:utm_zone': 30, 'mgrs:latitude_band': 'U', 'mgrs:grid_square': 'WF', 'grid:code': 'MGRS-30UWF', 'view:azimuth': 109.533382518936, 'view:incidence_angle': 10.712652049444431, 'view:sun_azimuth': 164.58766274291, 'view:sun_elevation': 14.254676794603498, 's2:tile_id': 'S2B_OPER_MSI_L2A_TL_2BPS_20240121T142720_A035913_T30UWF_N05.10', 's2:degraded_msi_data_percentage': 0, 's2:nodata_pixel_percentage': 76.185739, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0.009348, 's2:cl

Let's try to download publicly available bands in the Sentinel collection:

In [10]:
assets = item.assets
print(assets.keys())
print(assets["thumbnail"].href)

dict_keys(['red', 'green', 'blue', 'visual', 'nir', 'swir22', 'rededge2', 'rededge3', 'rededge1', 'swir16', 'wvp', 'nir08', 'scl', 'aot', 'coastal', 'nir09', 'cloud', 'snow', 'preview', 'granule_metadata', 'tileinfo_metadata', 'product_metadata', 'thumbnail'])
https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/30/U/WF/2024/1/S2B_T30UWF_20240121T111503_L2A/L2A_PVI.jpg


In [11]:
cloud = assets["cloud"]
if cloud is None:
    raise KeyError("The asset 'cloud' not found")
print(type(cloud))

print(cloud.href)   # URL to the asset
print(cloud.media_type)
print(cloud.roles) # roles might be non-intuitive
print(cloud.title)

<class 'pystac.asset.Asset'>
https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/30/U/WF/2024/1/S2B_T30UWF_20240121T111503_L2A/CLD_20m.tif
image/tiff; application=geotiff; profile=cloud-optimized
['data', 'cloud']
Cloud Probabilities


### EXPORT
First, we would like to export the full image locally to check it out:

In [12]:
import rioxarray
cloud_href = assets["cloud"].href
cloud = rioxarray.open_rasterio(cloud_href)
print(cloud)

<xarray.DataArray (band: 1, y: 5490, x: 5490)> Size: 30MB
[30140100 values with dtype=uint8]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 44kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y            (y) float64 44kB 6.1e+06 6.1e+06 6.1e+06 ... 5.99e+06 5.99e+06
    spatial_ref  int64 8B 0
Attributes:
    OVR_RESAMPLING_ALG:        AVERAGE
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        100
    STATISTICS_MEAN:           81.921746916104
    STATISTICS_MINIMUM:        1
    STATISTICS_STDDEV:         19.751638379983
    STATISTICS_VALID_PERCENT:  17.71
    _FillValue:                0
    scale_factor:              1.0
    add_offset:                0.0


Take a note that some scenes might have a small number of valid pixels (see **'STATISTICS_VALID_PERCENT'**). That doesn't mean these pixels are not suitable for follow-up analysis as this attribute describes the entire whole scene area whereas the actual area of interest might have larger share of valid pixels.

Moreover, no data values in this product usually mean that these pixels just belong to other non-cloudy categories (eg, vegetation, water, or snow).


In [13]:
# save whole image to the disk
cloud.rio.to_raster(f"data/cloud_{index}.tif")

Let's also download another band for a visual comparison with cloud probability - SCL, which divide scene by rough 'land-cover' categories, including clouds. As you can see, no data value (0) in the cloud probability product are usually observed in non-cloudy SCL categores, such as vegetation or water.

Therefore, no data value in a pixel doesn't mean we can't say for sure if it's a cloud or not - it usually means that it's not a cloud. In other words, `STATISTICS_VALID_PERCENT` is not a quality metric of cloud probability product and doesn't describe accuracy.

In [14]:
scl_href = assets["scl"].href
scl = rioxarray.open_rasterio(scl_href)
print(scl)

# save whole image to disk
scl.rio.to_raster(f"data/scl_{index}.tif")

# TODO - to add leaflet/folium for in-built visuals

<xarray.DataArray (band: 1, y: 5490, x: 5490)> Size: 30MB
[30140100 values with dtype=uint8]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 44kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y            (y) float64 44kB 6.1e+06 6.1e+06 6.1e+06 ... 5.99e+06 5.99e+06
    spatial_ref  int64 8B 0
Attributes:
    OVR_RESAMPLING_ALG:        MODE
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        11
    STATISTICS_MEAN:           8.2658636359963
    STATISTICS_MINIMUM:        2
    STATISTICS_STDDEV:         1.65467234972
    STATISTICS_VALID_PERCENT:  23.81
    _FillValue:                0
    scale_factor:              1.0
    add_offset:                0.0


Now, we would like to work with particular tiles. Let's call a separate function which will clip the scene by the extent of the tile of interest. 

In [15]:
import importlib # to reload external changes
import raster_utils

importlib.reload(raster_utils)

<module 'raster_utils' from '/app/raster_utils.py'>

Let's find out the National Grid tiles Tyne and Wear area intersects. For that purpose, we are going to use [20x20km grid](https://github.com/OrdnanceSurvey/OS-British-National-Grids?tab=readme-ov-file). Let's call the external function to find out the intersected tiles:

In [16]:
aoi_path = "data/NewcastleUponTyne.gpkg"
tile_path = "data/uk_20km_grid.gpkg"

touched, aoi_crs, tile_crs = raster_utils.touched_tiles(aoi_path, tile_path)
print(touched, aoi_crs, tile_crs )

Area of interest is EPSG:27700
    tile_name  country                                           geometry
413      NZ04  England  POLYGON ((400000 540000, 420000 540000, 420000...
414      NZ06  England  POLYGON ((400000 560000, 420000 560000, 420000...
418      NZ24  England  POLYGON ((420000 540000, 440000 540000, 440000...
419      NZ26  England  POLYGON ((420000 560000, 440000 560000, 440000...
423      NZ44  England  POLYGON ((440000 540000, 460000 540000, 460000...
424      NZ46  England  POLYGON ((440000 560000, 460000 560000, 460000... EPSG:27700 EPSG:27700


For now, we will limit calculations with one tile, mentioned above. Let's call the external function to clip the scene:

In [17]:
clipped_cloud=raster_utils.clip_scene_by_one_tile(cloud, touched, index=3)
clipped_cloud

<xarray.DataArray (band: 1, y: 648, x: 1000)> Size: 648kB
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)
Coordinates:
  * x            (x) float64 8kB 4.2e+05 4.2e+05 4.201e+05 ... 4.4e+05 4.4e+05
  * y            (y) float64 5kB 5.729e+05 5.729e+05 ... 5.6e+05 5.6e+05
  * band         (band) int64 8B 1
    spatial_ref  int64 8B 0
Attributes:
    OVR_RESAMPLING_ALG:        AVERAGE
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        100
    STATISTICS_MEAN:           81.921746916104
    STATISTICS_MINIMUM:        1
    STATISTICS_STDDEV:         19.751638379983
    STATISTICS_VALID_PERCENT:  17.71
    scale_factor:              1.0
    add_offset:                0.0
    _FillValue:                0

Again, you will see the **'STATISTICS_VALID_PERCENT'**, but this attribute inherited value from the initial image and is not correct anymore. If you wish, you can recalculate stats and find out how many non-cloudy pixels you have in your tile of interest now. It should be different from what you found out earlier in the metadata.

In [18]:
nodata_val=clipped_cloud.rio.nodata
print(f"Nodata value is {nodata_val}")
total_pixels=clipped_cloud.size
print(f"Total number of pixels is {total_pixels}")
nodata_pixels=((clipped_cloud == nodata_val).sum().item())
print(f"Number of pixels=0 is {nodata_pixels}")

nodata_share=(nodata_pixels/total_pixels)*100 if total_pixels >0 else 0
print(f"Share of pixels=0 is {nodata_share} %")

Nodata value is 0
Total number of pixels is 648000
Number of pixels=0 is 634242
Share of pixels=0 is 97.87685185185185 %


### SNOW PROBABILITY - NO DATA ISSUE

Let's check if we have the same problem with the similar product - snow probability. Since snow is not particularly common in the Tyne and Wear area, we’ll access a single scene where snow was known to be present.

In [19]:
target_id="S2B_T30UWF_20230116T111315_L2A" # another ID with snow - S2A_T30UWG_20230117T113415_L2A

items_by_id = {it.id: it for it in items}
item = items_by_id.get(target_id)
if item is None:
    raise ValueError(f"Item with id {target_id} not found")

print(item.id)
print(item.properties)

assets = item.assets
print(assets.keys())
print(assets["thumbnail"].href)

snow = assets["snow"]
if snow is None:
    raise KeyError("The asset 'snow' not found")
print(type(snow))

print(snow.href)   # URL to the asset
print(snow.media_type)
print(snow.roles) # roles might be non-intuitive
print(snow.title)

import rioxarray
snow_href = assets["snow"].href
snow = rioxarray.open_rasterio(snow_href)
print(snow)

# save whole image to disk
snow.rio.to_raster(f"data/snow_{target_id}.tif")

ValueError: Item with id S2B_T30UWF_20230116T111315_L2A not found

Note that snow probability product has the same issue. On the sample image you can see two types of pixels=0:
1. Pixels not covered by snow (these pixels are useful for further analysis)
2. Pixels outside of the satellite coverage, included into the grid, but divided from the snow pixels with a sharp straight line. These pixels are not useful for further analysis!

<img src="illustrations/nodata_issue_snow.png" alt="nodata_snow_issue" style="width:50%;">
<img src="illustrations/nodata_issue_snow_legend.png" alt="nodata_snow_issue_legend" style="width:30%;">

Now, let's export the output locally:

In [ ]:
clipped_cloud.rio.to_raster(f"data/clipped_cloud_{index}.tif")

#### CLEAN NO DATA VALUES
The problem is that there are two types of pixels with cloud probability encoded as 0:
1. within area covered by satellite, where there are no clouds. We want these pixels!
2. outside area covered by satellite, but included in the product image. We don't want these pixels!

For the valid calculations, we have to eliminate pixels of type 2.

To mask out the areas not covered by satellites, we will use SCL band. The main point is that SCL band doesn't have any no data values within the satellite-covered area. Therefore, we should consider pixels as true no-data values (**2**) only if they are marked as non-data in the SCL. Otherwise, these pixels are scanned by satellite, but not covered by clouds.

<img src="illustrations/scl_no_data.png" alt="scl_nodata" style="width:50%;">
<img src="illustrations/scl_no_data_legend.png" alt="scl_nodata_legend" style="width:30%;">


In [ ]:
# TODO - to write a function to separate no data values in the cloud probability product

### 2. CLOUD PROBABILITY CALCULATION **(ONE tile, ONE year)**

So far, everything was relatively simple, wasn't it?

Now, to understand how frequent you can enjoy sun (or suffer) in a particular pixel, we can try to extract the average value of cloud probability over a year. 

First, we could try it for only one tile of interest, for one year.
So, we will use all scenes (or timestamps, or items) available as we sure so far it won't take too long to calculate average values across <1000 of images.

**NOTE**: some images do not cover tiles entirely as satellites provide images in so-called swaths (MGRS grids), so swaths may slice the area of interest, partly leaving it without values.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

2.1. First, let's try just loop over all items. 
We could loop over all STAC items, open them, create one enormously giant array and calculate cloud probability, but that's definitely not the best day to do it.

So, we will test another way:
* loop over scenes (items)
* open first scene and extract value in each pixel
* store values in each pixel
* open next scene and extract value in each pixel
* add value in each pixel to previous value
* once all values across all scenes are cumulated, divide the cumulated value by the number of scenes

Just be aware before running cells, it may take quite a while.
Non-cached calculation for 2023 year (713 scenes) took 29 minutes (all tiles).

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# example: average cloud probability across all items
arrays = []
print ("Looping over all scenes (items):")

for i,item in enumerate(items): # STAC filtering
    cloud_asset = item.assets.get("cloud")
    if cloud_asset:
        da = rioxarray.open_rasterio(cloud_asset.href, masked=True)
        arrays.append(da)
        print(f"{i} - ID {item.id}")

# stack them along a new "time" dimension
print("Stacking items...")
stack = xr.concat(arrays, dim="time")
print(stack)

Looping over all scenes (items):


KeyboardInterrupt: 

Now, calculate the mean average for each pixel:

In [ ]:
mean_raster = stack.mean(dim="time", skipna=True)

NameError: name 'stack' is not defined

Export and visualise:

In [ ]:
mean_raster.rio.to_raster("data/cloud_mean.tif")
mean_raster.plot(figsize=(6,6), cmap="Blues")

In [ ]:
import numpy as np

mean_accum = None
count = 0
print (f"Looping over all scenes ({len(items)} items):")

for i,item in enumerate(items):
    
    cloud_asset = item.assets.get("cloud")
    if cloud_asset:
        da = rioxarray.open_rasterio(cloud_asset.href)
        data = da.squeeze().values.astype("float32")

        if mean_accum is None:
            mean_accum = np.zeros_like(data)
            ref_da = da  # keep reference for spatial metadata

        mean_accum += data
        # print(mean_accum)
        count += 1
        print(f"{i} - ID {item.id}")

mean_accum /= count # calculate average # it's numpy ndarray

Looping over all scenes (64 items):
0 - ID S2A_T30UWF_20230131T111307_L2A


In [ ]:
print(mean_accum)

Export and visualise the output:

In [ ]:
import rasterio
import matplotlib.pyplot as plt

transform = ref_da.rio.transform()
crs = ref_da.rio.crs
height, width = ref_da.shape[-2:]

profile = {
    "driver": "GTiff",
    "dtype": "float32",
    "count": 1,
    "height": height,
    "width": width,
    "crs": crs,
    "transform": transform,
    "compress": "lzw"
}

with rasterio.open("data/cloud_mean.tif", "w", **profile) as dst:
    dst.write(mean_accum, 1)

print("✅ Exported mean raster to data/cloud_mean.tif")

# to drop the 'band' dimension (not needed, we have only one possible band)
# wrap numpy array with spatial metadata from referenece
mean_da = ref_da.squeeze().copy(data=mean_accum)

mean_da.plot(
    figsize=(6, 6),
    cmap="Blues",
    cbar_kwargs={"label": "Mean cloud probability (%)"}
)
plt.title("Mean yearly cloud probability")
plt.show()

NameError: name 'ref_da' is not defined

Takes an enormous amount of time (if your Notebook didn't crash), isn't it? And that's all just for one year.

### TILE OF INTEREST
We can test the performance for our tile of interest. Let's define parameters:


In [20]:
%reload_ext autoreload
%autoreload 2

# parameters
aoi_path = "data/NewcastleUponTyne.gpkg"
tile_path = "data/uk_20km_grid.gpkg"

collection="sentinel-2-c1-l2a"
datetime='2024-01-01/2024-12-31'
asset="cloud"

touched, aoi_crs, tile_crs = raster_utils.touched_tiles(aoi_path, tile_path)
print(touched)
spatial_extent = touched.iloc[[3]].copy()  # double brackets to keep as DataFrame
print(spatial_extent) # THAT'S OUR TILE
print(type(spatial_extent)) 

Area of interest is EPSG:27700
    tile_name  country                                           geometry
413      NZ04  England  POLYGON ((400000 540000, 420000 540000, 420000...
414      NZ06  England  POLYGON ((400000 560000, 420000 560000, 420000...
418      NZ24  England  POLYGON ((420000 540000, 440000 540000, 440000...
419      NZ26  England  POLYGON ((420000 560000, 440000 560000, 440000...
423      NZ44  England  POLYGON ((440000 540000, 460000 540000, 460000...
424      NZ46  England  POLYGON ((440000 560000, 460000 560000, 460000...
    tile_name  country                                           geometry
419      NZ26  England  POLYGON ((420000 560000, 440000 560000, 440000...
<class 'geopandas.geodataframe.GeoDataFrame'>


Now, calculate cloud probability, masking the scenes with the tile of interest. We avoid loading the full raster into memory.

**ATTENTION**: if processing is too slow, consider the reduced time period, eg `2024-01-01/2024-01-31`

In [21]:
import numpy as np
import rasterio
from rasterio.windows import from_bounds
from rasterio.transform import rowcol
from rasterio.warp import reproject, Resampling, calculate_default_transform
import geopandas as gpd

mean_accum=None
count = 0
ref_profile=None
print(f"Looping over all scenes ({len(items)} items):")
print("-" * 40)

spatial_extent.to_file("data/spatial_extent.gpkg", driver="GPKG")

for i,item in enumerate(items):
    cloud_asset = item.assets.get("cloud")
    if cloud_asset:
        # da = da.rio.clip(spatial_extent.geometry, spatial_extent.crs, drop=False)  # NOTE: another masking option (still loads everything to memory)
        with rasterio.open(cloud_asset.href) as src:            
            '''window = from_bounds(*spatial_extent.total_bounds, transform=src.transform)'''

            data=src.read(1).astype("float32")

            print(f"Data shape: {data.shape}")

            """# AOI bounds in raster CRS
            minx, miny, maxx, maxy = spatial_extent.total_bounds
            print(minx,miny,maxx,maxy)
            print(spatial_extent.crs)
            print(src.crs)"""

            # REPROJECTION
            if spatial_extent.crs != src.crs: 
                transform, width, height = calculate_default_transform(
                    src.crs, spatial_extent.crs, src.width, src.height, *src.bounds
                )
                data_reproj = np.empty((height, width), dtype=np.float32)
                
                """DEBUG
                print (data_reproj.shape)
                print(width, height)"""

                reproject(
                    source=rasterio.band(src, 1),
                    destination=data_reproj,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=spatial_extent.crs,
                    resampling=Resampling.bilinear #NOTE:bilinear?
                )

                data = data_reproj
                print(f"Data reprojected shape: {data.shape}")

            else:
                data = src.read(1).astype("float32")
                transform = src.transform
                #NOTE: crucial because we need cartesian CRS (recorded in tile, not in Sentinel scenes)


            # clip to spatial extent using pixel indices--
            minx, miny, maxx, maxy = spatial_extent.total_bounds

            row_min, col_min = rowcol(transform, minx, maxy)  # upper-left
            row_max, col_max = rowcol(transform, maxx, miny)  # lower-right
            # get indices within array bounds
            row_min = max(row_min, 0)
            col_min = max(col_min, 0)
            row_max = min(row_max, data.shape[0]-1)
            col_max = min(col_max, data.shape[1]-1)

            # slice array
            data_clipped = data[row_min:row_max+1, col_min:col_max+1].astype("float32")

            '''DEBUG 
            print(row_min,col_min,row_max,col_max)'''

            print(f"Data clipped shape: {data_clipped.shape}")

            # NOTE: we moved from rasterio window because it works only with the rasters of the same CRS. Otherwise, we would need to store temporary reprojected rasters in memory

            # NOTE: DEBUG
            """
            # AOI bounds in raster CRS
            minx, miny, maxx, maxy = spatial_extent.total_bounds
            print(minx,miny,maxx,maxy)
            # raster resolution (pixel size)
            res_x, res_y = src.res
            width = int((maxx - minx) / res_x)
            height = int((maxy - miny) / res_y)
            print(f"AOI width: {width}, height: {height}, resolution: {res_x}")
            """

            print(f"{i} - ID {item.id}, shape: {data_clipped.shape}, bounds: {spatial_extent.total_bounds}")
            print("-"*40)
            
            # initialize reference profile
            if ref_profile is None:
                ref_profile = src.profile.copy()
                ref_profile.update({
                    "height": data_clipped.shape[0],
                    "width": data_clipped.shape[1],
                    "transform": rasterio.transform.from_bounds(minx, miny, maxx, maxy,
                                                               data_clipped.shape[1],
                                                               data_clipped.shape[0]),
                    "dtype": "float32",
                    "crs": spatial_extent.crs,
                    "count": 1,
                    "compress": "lzw"
                })
                mean_accum = np.zeros_like(data_clipped)

                print (f"DEBUG: reference profile is saved for CRS:{spatial_extent.crs}")
                print("-" * 40)
                'valid_count = np.zeros_like(data) # TODO - to consider later when no data values and true ZEROs are separated'

            mean_accum += data_clipped #TODO - shape of arrays is different because they can cover the AOI only partially. Resample to the area of tile?
            count += 1
        
        if mean_accum is None:
            mean_accum = np.zeros_like(data)

    
mean_accum /= count # calculate average # it's numpy ndarray
print(f"Processed {len(items)} scenes. Mean raster shape: {mean_accum.shape}")

Looping over all scenes (64 items):
----------------------------------------
Data shape: (5490, 5490)
Data reprojected shape: (5568, 5568)
Data clipped shape: (648, 1001)
0 - ID S2B_T30UWF_20240131T111211_L2A, shape: (648, 1001), bounds: [420000. 560000. 440000. 580000.]
----------------------------------------
DEBUG: reference profile is saved for CRS:EPSG:27700
----------------------------------------
Data shape: (5490, 5490)
Data reprojected shape: (5568, 5568)
Data clipped shape: (577, 269)
1 - ID S2B_T30UXF_20240131T111211_L2A, shape: (577, 269), bounds: [420000. 560000. 440000. 580000.]
----------------------------------------


ValueError: operands could not be broadcast together with shapes (648,1001) (577,269) (648,1001) 

In [22]:
import numpy as np
import rioxarray
import geopandas as gpd
from shapely.geometry import box
import xarray as xr

spatial_extent.to_file("data/spatial_extent.gpkg", driver="GPKG") # export spatial extent to a file

def check_raster_covers_aoi(spatial_extent: gpd.GeoDataFrame, asset_href:str ) -> xr.DataArray | None:
    """
    Parameters:
        spatial_extent (gpd.GeoDataFrame): prepared area of interest (AOI)
        asset_href (str): name of the STAC asset (eg., "cloud" or "snow) to open
    Returns:
        xr.DataArray or None: if raster fully covers AOI, false otherwise

    Note: this function doesn't open the raster.
    """
    raster = rioxarray.open_rasterio(asset_href).squeeze()

    # reproject AOI to raster's CRS NOTE: more lightweight, compared to raster's reprojection to AOI
    aoi_reproj = spatial_extent.to_crs(raster.rio.crs)
    # TODO - to create a function to extract CRS from raster and reproject AOI to raster's CRS (just once, without a loop, to speed up)

    # creating bboxes
    aoi_bbox = box(*aoi_reproj.total_bounds)
    raster_bbox = box(*raster.rio.bounds())

    print("AOI CRS:", spatial_extent.crs)
    print("Raster CRS:", raster.rio.crs)
    print("AOI bounds (in raster CRS):", aoi_bbox.bounds)
    print("Raster bounds:", raster_bbox.bounds)
    print("-" * 40)

    if raster_bbox.contains(aoi_bbox):
        return raster
    else:
        return None

def create_reference_raster(spatial_extent: gpd.GeoDataFrame,asset_href: str, res: float) -> xr.DataArray:
    """
    Create a reference raster matching the AOI extent and specified resolution (from the first item of STAC collection)

    Parameters:
        spatial_extent (gpd.GeoDataFrame): AOI
        asset_href (str): raster to use as reference
        resolution (float, optional): desired spatial resolution (pixel size). If None, use raster's resolution.
    
    Returns:
        ref_raster (xr.DataArray): reference raster with correct extent abd resolution
    """
    raster = rioxarray.open_rasterio(asset_href).squeeze()

    # reproject AOI to raster's CRS
    aoi_reproj = spatial_extent.to_crs(raster.rio.crs)
    xmin, ymin, xmax, ymax = aoi_reproj.total_bounds

    print(xmin, ymin, xmax, ymax)

    #bboxes
    aoi_bbox = box(*aoi_reproj.total_bounds)
    raster_bbox = box(*raster.rio.bounds())

    print(aoi_bbox,raster_bbox)

    # use raster's resolution if none provided
    if res is None:
        res_x, res_y = raster.rio.resolution()
        res = abs(res_x)  # take magnitude
    else:
        res_x = res_y = res

    print(res_x, res_y, res)
    
    x = np.arange(xmin + res/2, xmax, res)
    y = np.arange(ymax - res/2, ymin, -res) # top-to-bottom

    # dataarray filled with zeros
    ref_raster= xr.DataArray(
        np.zeros((len(y), len(x)), dtype=np.float32),
        coords={"y": y, "x": x},
        dims=("y", "x"),
    )

    ref_raster.attrs["crs"] = aoi_reproj.crs

    return ref_raster

# 1.Ffnd reference raster that fully covers AOI (loop over all rasters)
ref_raster = None
for item in items:
    cloud_asset = item.assets.get("cloud")
    if not cloud_asset:
        continue
    
    ref_raster = check_raster_covers_aoi(spatial_extent=spatial_extent, asset_href=cloud_asset.href)
    if ref_raster is not None:
        print("Found raster fully covering AOI")
        break  # stop at the first one

# 2.create the reference raster if not found amongst rasters
if ref_raster is None:
    print("No rasters fully cover the AOI, creating the reference raster...")
    first_raster_href = items[0].assets["cloud"].href
    ref_raster = create_reference_raster(spatial_extent=spatial_extent, asset_href=first_raster_href, res=None)
    print(ref_raster)

    output_path = "data/ref_raster.tif"
    ref_raster.rio.to_raster(output_path)


    # NOTE: TODO - to write a function for creating the reference raster if we can't find it amongst the rasters

"""
# STEP 2 — Reproject reference raster to AOI CRS and clip to AOI
ref_raster = ref_raster.rio.reproject(aoi_crs)
ref_raster_clipped = ref_raster.rio.clip(spatial_extent.geometry, aoi_crs, drop=False)

mean_accum = np.zeros(ref_raster_clipped.shape, dtype=np.float32)

print(f"Reference raster shape: {ref_raster_clipped.shape}")
print("-" * 40)

# STEP 3 — Process all rasters aligned to reference
for i, item in enumerate(items):
    cloud_asset = item.assets.get("cloud")
    if not cloud_asset:
        continue

    da = rioxarray.open_rasterio(cloud_asset.href, masked=True).squeeze()
    da_reproj = da.rio.reproject_match(ref_raster_clipped)
    da_clipped = da_reproj.rio.clip(spatial_extent.geometry, aoi_crs, drop=False)

    mean_accum += da_clipped.fillna(0).values
    count += 1

    print(f"{i} - ID {item.id}, shape after clip: {da_clipped.shape}")

# STEP 4 — Compute and export mean
mean_accum /= count
mean_da = ref_raster_clipped.copy(data=mean_accum)
mean_da.rio.to_raster("data/mean_cloud_probability.tif", compress="LZW")

print(f"\n✅ Processed {count} rasters — saved mean raster to data/mean_cloud_probability.tif")
"""


AOI CRS: EPSG:27700
Raster CRS: EPSG:32630
AOI bounds (in raster CRS): (583684.9425964617, 6088264.899489767, 603970.3792341521, 6108550.197571301)
Raster bounds: (499980.0, 5990220.0, 609780.0, 6100020.0)
----------------------------------------
AOI CRS: EPSG:27700
Raster CRS: EPSG:32630
AOI bounds (in raster CRS): (583684.9425964617, 6088264.899489767, 603970.3792341521, 6108550.197571301)
Raster bounds: (600000.0, 5990220.0, 709800.0, 6100020.0)
----------------------------------------
AOI CRS: EPSG:27700
Raster CRS: EPSG:32630
AOI bounds (in raster CRS): (583684.9425964617, 6088264.899489767, 603970.3792341521, 6108550.197571301)
Raster bounds: (499980.0, 6090240.0, 609780.0, 6200040.0)
----------------------------------------
AOI CRS: EPSG:27700
Raster CRS: EPSG:32630
AOI bounds (in raster CRS): (583684.9425964617, 6088264.899489767, 603970.3792341521, 6108550.197571301)
Raster bounds: (600000.0, 6090240.0, 709800.0, 6200040.0)
----------------------------------------
AOI CRS: EPS

'\n# STEP 2 — Reproject reference raster to AOI CRS and clip to AOI\nref_raster = ref_raster.rio.reproject(aoi_crs)\nref_raster_clipped = ref_raster.rio.clip(spatial_extent.geometry, aoi_crs, drop=False)\n\nmean_accum = np.zeros(ref_raster_clipped.shape, dtype=np.float32)\n\nprint(f"Reference raster shape: {ref_raster_clipped.shape}")\nprint("-" * 40)\n\n# STEP 3 — Process all rasters aligned to reference\nfor i, item in enumerate(items):\n    cloud_asset = item.assets.get("cloud")\n    if not cloud_asset:\n        continue\n\n    da = rioxarray.open_rasterio(cloud_asset.href, masked=True).squeeze()\n    da_reproj = da.rio.reproject_match(ref_raster_clipped)\n    da_clipped = da_reproj.rio.clip(spatial_extent.geometry, aoi_crs, drop=False)\n\n    mean_accum += da_clipped.fillna(0).values\n    count += 1\n\n    print(f"{i} - ID {item.id}, shape after clip: {da_clipped.shape}")\n\n# STEP 4 — Compute and export mean\nmean_accum /= count\nmean_da = ref_raster_clipped.copy(data=mean_accum)\

Surprisingly, the time required is comparable to previous calculation because computational performance probably depends mostly on the loading scenes, not on calculating the cloud probability. Still, we have to download the full scenes, and calculating the cloud probability in the smaller area doesn't help much to reduce the computation time.

In [23]:

"""
transform = ref_da.rio.transform()
crs = ref_da.rio.crs
height, width = ref_da.shape[-2:]
"""
"""
# define specifications for export from the spatial extent
res_x, res_y = src.res
width = int((maxx - minx) / res_x)
height = int((maxy - miny) / res_y)
print(f"AOI width: {width}, height: {height}, resolution: {res_x}")

crs = spatial_extent.crs

profile = {
    "driver": "GTiff",
    "dtype": "float32",
    "count": 1,
    "height": height,
    "width": width,
    "crs": crs,
    "compress": "lzw"
}

print(crs)
"""

with rasterio.open("data/cloud_mean_bbox.tif", "w",**ref_profile) as dst:
    dst.write(mean_accum, 1)

print("Exported mean raster to data/cloud_mean_bbox.tif")

# to drop the 'band' dimension (not needed, we have only one possible band)
# wrap numpy array with spatial metadata from referenece

# resize mean_accum to ref_da shape
mean_accum = mean_accum.reshape(src.shape)

mean_accum_resized = mean_accum[:ref_da.shape[0], :ref_da.shape[1]]

mean_da = ref_da.copy(data=mean_accum_resized)


mean_da.plot(
    figsize=(6, 6),
    cmap="Blues",
    cbar_kwargs={"label": "Mean cloud probability (%), clipped by a single tile"}
)
plt.title("Mean yearly cloud probability")
plt.show()

Exported mean raster to data/cloud_mean_bbox.tif


ValueError: cannot reshape array of size 648648 into shape (5490,5490)

Upon checking the results, we can spot some issues in data product with buildings roofs - some warehouses and industrial scenes with white roofs tend to have the higher cloud probability.

<img src="illustrations/issue_roofs.png" alt="roofs_issue" style="width:80%;">

#### TODO - RASTER STATS

It would be nice to provide a quick statistical analysis of cloud probability distribution. For example, range of values is not large because clouds persistently cover all of the pixels - there are no regions where clouds are very rare (at first glance)

2.2. Local analysis

LSOA, MSOA level...
To figure out how things work out locally, we will use the super output areas data available from the [Geographical Data Service](https://data.geods.ac.uk/dataset/lsoac).
Let's download it and find the LSOAs intersected by our AOI.

In [24]:
# NOTE: this dataset is not available on GitHub, please download it from the project Sharepoint or GeoDS
lsoa="data/LSOA_2021_EW_BSC_V4.shp"

lsoa_df=gpd.read_file(lsoa)
aoi=gpd.read_file(aoi_path)

def lsoa_by_aoi(df: gpd.GeoDataFrame, aoi: gpd.GeoDataFrame, out_path: str) -> gpd.GeoDataFrame:
    """This function filters LSOAs intersected by the area of interest (keeps geometry untouched)
    Parameters:
    df (gpd.GeoDataFrame): LSOA geometries
    aoi (gpd.GeoDataFrame): geometries of area of interest
    out_path (str): path to the output (intersectin=on)

    Returns:
    gpd.GeoDataFrame: LSOAs intersected with the area of interest
    """

    if df.crs!=aoi.crs:
        print("CRS of AOI and LSOA do not comply. Reprojecting AOI...")
        aoi=aoi.to_crs(df.crs)
    
    lsoa_in_aoi=df[df.intersects(aoi.unary_union)] # `unary_union`` deprecated, but `union_all` doesn't work in this gpd version
    print(f"{len(lsoa_in_aoi)} LSOAs intersect with the area of interest")
    
    lsoa_in_aoi.to_file(out_path,driver="GPKG")
    return lsoa_in_aoi

lsoa_in_aoi=lsoa_by_aoi(lsoa_df, aoi, out_path="data/lsoa_intersected.gpkg")

756 LSOAs intersect with the area of interest


/tmp/ipykernel_55669/1254428640.py:22: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  lsoa_in_aoi=df[df.intersects(aoi.unary_union)] # `unary_union`` deprecated, but `union_all` doesn't work in this gpd version


Now, we would like to know what is the average cloud probability in each LSOA because raster product is not as representative as LSOAs for local planning. For the area of interest let's use the Newcastle border first.

In [25]:
from rasterstats import zonal_stats
import tempfile
import warnings

#reload cloud probability product
cloud_mean_bbox=rasterio.open("data/cloud_mean_bbox.tif")
cloud_mean_bbox_xr = rioxarray.open_rasterio(cloud_mean_bbox, mask_and_scale=False)

cloud_mean=rasterio.open("data/cloud_mean.tif")
cloud_mean_xr = rioxarray.open_rasterio(cloud_mean, mask_and_scale=False)

def extract_mean_per_lsoa(lsoa: gpd.GeoDataFrame, raster: xr.DataArray) -> gpd.GeoDataFrame:
    """ This function extracts the average value of cloud probability per LSOA and counts number of pixels used to count the cloud probability
    Parameters:
        lsoa (gpd.GeoDataFrame): LSOA geodataframe in our area of interest
        raster (xr.DataArray): raster with mean cloud probability and pixel count
    Returns:
        gpd.GeoDataFrame: the LSOAs geodataframe with added columns
    """

    if lsoa.crs !=raster.rio.crs:
        raster = raster.rio.reproject(lsoa.crs)
        '''raise ValueError("CRS of raster data and LSOA do not match. Check previous steps!")'''
        
    # write raster to a temporary file
    with tempfile.NamedTemporaryFile(suffix=".tif") as tmp:
        raster.rio.to_raster(tmp.name)
        stats = zonal_stats(
            vectors=lsoa,
            raster=tmp.name,
            stats=["mean", "count"],
            nodata=raster.rio.nodata
        ) # NOTE: takes ~4s for whole Newcastle, 1 year, ~700 LCAs, one big raster (grid tile)

    
    lsoa=lsoa.copy()
    lsoa["mean_cloud"] = [s["mean"] for s in stats]
    lsoa["pixel_count"] = [s["count"] for s in stats]

    # check if there are any LSOAs not covered by raster values
    no_pixels=[s["count"]==0 or s["mean"] is None for s in stats]
    if any(no_pixels):
        warnings.warn(f"{sum(no_pixels)} LSOAs don't have any values or pixels in the correspondent raster. Please check the input raster dataset")

    return lsoa

lsoa_stats=extract_mean_per_lsoa(lsoa_in_aoi,raster=cloud_mean_xr)

/tmp/ipykernel_55669/675782027.py:43: UserWarning: 31 LSOAs don't have any values or pixels in the correspondent raster. Please check the input raster dataset
  warnings.warn(f"{sum(no_pixels)} LSOAs don't have any values or pixels in the correspondent raster. Please check the input raster dataset")


In [29]:
#print(lsoa_stats)
lsoa_stats.to_file("data/lsoa_stats_city.gpkg", driver="GPKG")

Let's use now a tile as an area of interest.

In [28]:
lsoa_in_aoi=lsoa_by_aoi(lsoa_df, spatial_extent, out_path="data/lsoa_tile_intersected.gpkg")

lsoa_stats=extract_mean_per_lsoa(lsoa_in_aoi,raster=cloud_mean_xr)
# print(lsoa_stats)
lsoa_stats.to_file("data/lsoa_tile.gpkg", driver="GPKG")

/tmp/ipykernel_55669/1254428640.py:22: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  lsoa_in_aoi=df[df.intersects(aoi.unary_union)] # `unary_union`` deprecated, but `union_all` doesn't work in this gpd version


523 LSOAs intersect with the area of interest


/tmp/ipykernel_55669/675782027.py:43: UserWarning: 71 LSOAs don't have any values or pixels in the correspondent raster. Please check the input raster dataset
  warnings.warn(f"{sum(no_pixels)} LSOAs don't have any values or pixels in the correspondent raster. Please check the input raster dataset")


#### 2.3. Parallelised calculation

# TODO - Test Dask locally for CPU parallelisation

#### DEPRECATED - Sentinel Hub API (Process API)

~~Process API requires Sentinel Hub account. Please check [configuration instructions](https://sentinelhub-py.readthedocs.io/en/latest/configure.html) about how to set up your Sentinel Hub credentials.~~

In [ ]:
from sentinelhub import SHConfig
"""
from oauthlib.oauth2 import BackendApplicationClient
# it's recommended to install sentinelhub[AWS]: pip install sentinelhub[AWS]
from requests_oauthlib import OAuth2Session
config = SHConfig()


CLIENT_ID = "..."
CLIENT_SECRET = "..."
TOKEN_URL = "https://services.sentinel-hub.com/auth/realms/main/protocol/openid-connect/token"

# set up credentials
client = BackendApplicationClient(client_id=CLIENT_ID)
oauth = OAuth2Session(client=client)

# get an authentication token
token = oauth.fetch_token(
    token_url=TOKEN_URL,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)
"""

"""
if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")

if config.sh_client_id and config.sh_client_secret:
    print("config found")

print(config)

# to check where the configuration is 
SHConfig.get_config_location()"""

# TODO - to find out how adjust config.toml file

DEPRECATED - EARTH DAILY:

~~Let's also try Earth Daily STAC:
ACCOUNT TO BE CREATED YET (https://console.earthdaily.com/platform/signin)~~

In [ ]:
"""import os
import requests

from dotenv import load_dotenv
from pystac.client import Client

load_dotenv()  # take environment variables from .env.

CLIENT_ID = os.getenv("EDS_CLIENT_ID")
CLIENT_SECRET = os.getenv("EDS_SECRET")
EDS_AUTH_URL = os.getenv("EDS_AUTH_URL")
API_URL = os.getenv("EDS_API_URL")
STAC_API_URL = f"{API_URL}/platform/v1/stac"

# Setup requests session
session = requests.Session()
session.auth = (CLIENT_ID, CLIENT_SECRET)
"""

"""
def get_new_token(session):
    '''Obtain a new authentication token using client credentials.'''
    token_req_payload = {"grant_type": "client_credentials"}
    try:
        token_response = session.post(EDS_AUTH_URL, data=token_req_payload)
        token_response.raise_for_status()
        tokens = token_response.json()
        return tokens["access_token"]
    except requests.exceptions.RequestException as e:
        print(f"Failed to obtain token: {e}")

token = get_new_token(session)

client = Client.open(STAC_API_URL, headers={"Authorization": f"bearer {token}"}) 
"""